In [1]:
%pip install mne numpy scipy scikit-learn tensorflow keras colorlog shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.2/540.2 kB 14.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import utils
import mne
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

logger = utils.get_logger()

2024-08-15 15:34:53.601620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 15:34:53.651191: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 15:34:53.670043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 15:34:53.715312: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 15:34:56.605461: W tensorflow/comp

# Neural networks
This notebook explores the usability of neural network architectures on the cleaned rsEEG data. This doesn't make use of the extracted features, which means that there is a larger chance of finding patterns in the raw data and might result in better generalizability to other patients.

In [3]:
labels_df = utils.get_metadata_df("dataset-cleaned", "Randomisatielijst.csv")
# patient_ids = ["02", "04", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
patient_ids = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
train_patients, test_patients = train_test_split(patient_ids, test_size=0.2, random_state=42)

In [4]:
filtered_df = labels_df[
    (labels_df['eeg_type'] == 'rsEEG')
    & (labels_df['procedure'] == 'itbs')
    & (labels_df['patient_id'].isin(train_patients))]
epoch_files = filtered_df['filename'].tolist()
epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in epoch_files]
all_epochs = mne.concatenate_epochs(epochs_list)

# Training data & labels
X_1D = all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
y_1D = np.array([0 if timing == 'pre' else 1 for timing in filtered_df['pre_post']])

# Fix length of data with labels
if len(X_1D) != len(y_1D):
    y_1D = np.repeat(y_1D, len(X_1D) // len(y_1D) + 1)[:len(X_1D)]
if len(X_1D) != len(y_1D):
    raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X_1D)}, y sizes: {len(y_1D)}")

Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_14_S2_rsEEG_post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
274 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_11_S1_rsEEG_pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
255 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_18_S3_rsEEG_pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Reading

/tmp/ipykernel_3792/2350113372.py:7: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
4073 matching events found
Applying baseline correction (mode: mean)


/tmp/ipykernel_3792/2350113372.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_1D = all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


In [7]:
model_1D = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_1D.shape[1], X_1D.shape[2])),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Conv1D(256, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_1D.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1D.fit(X_1D, y_1D, epochs=30, batch_size=32)

/opt/conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-15 15:36:48.698264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:3b:00.0, compute capability: 6.1


Epoch 1/30


I0000 00:00:1723736226.439738   12696 service.cc:146] XLA service 0x7efea0086a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723736226.439842   12696 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-08-15 15:37:06.592260: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-15 15:37:07.409113: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  4/128 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.4251 - loss: 0.7572 

I0000 00:00:1723736238.073719   12696 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


128/128 ━━━━━━━━━━━━━━━━━━━━ 31s 97ms/step - accuracy: 0.4847 - loss: 0.7900
Epoch 2/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5017 - loss: 0.7064
Epoch 3/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5179 - loss: 0.6971
Epoch 4/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.4968 - loss: 0.6938
Epoch 5/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5188 - loss: 0.6953
Epoch 6/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5348 - loss: 0.6877
Epoch 7/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5230 - loss: 0.6864
Epoch 8/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5517 - loss: 0.6838
Epoch 9/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6062 - loss: 0.6679
Epoch 10/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5961 - loss: 0.6609
Epoch 11/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6192 - loss: 0.6501
Epoch 12/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/st

In [8]:
total_accuracy = []
for new_patient_id in test_patients:
    logger.info(f"Testing on patient {new_patient_id}")
    new_filtered_df = labels_df[(labels_df['procedure'] == 'itbs') & (labels_df['eeg_type'] == 'rsEEG') & (labels_df['patient_id'] == new_patient_id)]
    new_epoch_files = new_filtered_df['filename'].tolist()
    new_epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in new_epoch_files]
    new_all_epochs = mne.concatenate_epochs(new_epochs_list)
    
    X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    y_test = np.array([0 if timing == 'pre' else 1 for timing in new_filtered_df['pre_post']])
    
    # Ensure y has the same number of samples as X
    if len(X_test) != len(y_test):
        y_test = np.repeat(y_test, len(X_test) // len(y_test) + 1)[:len(X_test)]
    
    # Check if the lengths match
    if len(X_test) != len(y_test):
        raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X_test)}, y sizes: {len(y_test)}")
    
    y_pred = model_1D.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int).flatten()
    
    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    logger.info(f'Accuracy for patient {new_patient_id}: {accuracy}')
    total_accuracy.append(accuracy)

average_accuracy = np.mean(total_accuracy)
logger.info(f'Total accuracy (averaged): {average_accuracy}')

[2024-08-15 15:38:33,741] - INFO - Testing on patient 13


Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_13_S2_rsEEG_pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_13_S2_rsEEG_post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
346 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_3792/398768274.py:7: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  new_all_epochs = mne.concatenate_epochs(new_epochs_list)


Not setting metadata
644 matching events found
Applying baseline correction (mode: mean)


/tmp/ipykernel_3792/398768274.py:9: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


...

/tmp/ipykernel_3792/398768274.py:7: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  new_all_epochs = mne.concatenate_epochs(new_epochs_list)


Not setting metadata
558 matching events found
Applying baseline correction (mode: mean)


/tmp/ipykernel_3792/398768274.py:9: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


[2024-08-15 15:38:54,067] - INFO - Accuracy for patient 17: 0.34587813620071683
[2024-08-15 15:38:54,070] - INFO - Total accuracy (averaged): 0.4402394654150328


## 2D Conv

In [5]:
# 2x2 and 3x3 kernel have same result

X_2D = X_1D
y_2D = y_1D
X_CNN = X_2D.reshape(X_2D.shape[0], X_2D.shape[1], X_2D.shape[2], 1)  # Shape: (n_epochs, n_channels, n_times, 1)

model_CNN = Sequential([
    Conv2D(64, (2, 2), activation='relu', input_shape=(X_CNN.shape[1], X_CNN.shape[2], X_CNN.shape[3])),
    BatchNormalization(),
    #MaxPooling2D((2, 2)),
    #Dropout(0.25),
    
    Conv2D(128, (2, 2), activation='relu'),
    BatchNormalization(),
    #MaxPooling2D((2, 2)),
    #Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_CNN.fit(X_CNN, y_2D, epochs=10, batch_size=32)

/opt/conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-14 13:06:09.492002: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2024-08-14 13:06:19.746456: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.32GiB (rounded to 7860387840)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summa

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

In [ ]:
total_accuracy = []
for new_patient_id in test_patients:
    logger.info(f"Testing on patient {new_patient_id}")
    new_filtered_df = labels_df[(labels_df['procedure'] == 'itbs') & (labels_df['eeg_type'] == 'rsEEG') & (labels_df['patient_id'] == new_patient_id)]
    new_epoch_files = new_filtered_df['filename'].tolist()
    new_epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in new_epoch_files]
    new_all_epochs = mne.concatenate_epochs(new_epochs_list)
    
    X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    y_test = np.array([0 if timing == 'pre' else 1 for timing in new_filtered_df['pre_post']])
    
    # Ensure y has the same number of samples as X
    if len(X_test) != len(y_test):
        y_test = np.repeat(y_test, len(X_test) // len(y_test) + 1)[:len(X_test)]
    
    # Check if the lengths match
    if len(X_test) != len(y_test):
        raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X_test)}, y sizes: {len(y_test)}")
    
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int).flatten()
    
    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    logger.info(f'Accuracy for patient {new_patient_id}: {accuracy}')
    total_accuracy.append(accuracy)

average_accuracy = np.mean(total_accuracy)
logger.info(f'Total accuracy (averaged): {average_accuracy}')

## XAI

In [ ]:
import shap
import numpy as np

explainer = shap.DeepExplainer(1D_model, X_1D[:100])
shap_values = explainer.shap_values(X_test[:10])
shap.summary_plot(shap_values, X_test)


total_accuracy = []
for new_patient_id in test_patients:
    logger.info(f"Testing on patient {new_patient_id}")
    new_filtered_df = labels_df[(labels_df['procedure'] == 'itbs') & (labels_df['eeg_type'] == 'rsEEG') & (labels_df['patient_id'] == new_patient_id)]
    new_epoch_files = new_filtered_df['filename'].tolist()
    new_epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in new_epoch_files]
    new_all_epochs = mne.concatenate_epochs(new_epochs_list)
    
    X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    y_test = np.array([0 if timing == 'pre' else 1 for timing in new_filtered_df['pre_post']])
    
    # Ensure y has the same number of samples as X
    if len(X_test) != len(y_test):
        y_test = np.repeat(y_test, len(X_test) // len(y_test) + 1)[:len(X_test)]
    
    # Check if the lengths match
    if len(X_test) != len(y_test):
        raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X_test)}, y sizes: {len(y_test)}")